# kiwipiepy 모듈
```
pip install kiwipiepy
```
jupyter-notebook 단축기
* `a` : 위에 셀 추가
* `b` : 아래에 셀 추가
* `dd` : 셀 삭제
* `m` : 문서 셀로 변경
* `y` : 코드 셀로 변경

In [28]:
file = open("한국어불용어100.txt", 'r', encoding="utf-8")
stopword=[]

while True:
    line = file.readline()
    if not line: break
    wordlist = line.split('\t')
    if (wordlist[1].startswith('N')):
        stopword.append(wordlist[0])

print(stopword)

['것', '수', '이', '나', '사람', '등', '우리', '때', '년', '말', '일', '때문', '그것', '일', '문제', '사회', '중', '씨', '지금', '속', '하나', '집', '월', '데', '자신', '내', '경우', '명', '생각', '시간', '그녀', '앞', '번', '여자', '개', '전', '사실', '점', '정도', '원', '소리']


In [12]:
class Reader:
    def __init__(self, filePath):
        self.file = open(filePath,'r',encoding='UTF-8')
    
    def read(self, id):
        if id == 0: self.file.seek(0)
        return self.file.readline()

reader = Reader('datas.txt')

In [14]:
import sys
import io
import re # 정규표현식 패키지
from kiwipiepy import Kiwi, Option
import tomotopy as tp

file = open("한국어불용어100.txt", 'r', encoding="utf-8")
stopword=[]

while True:
    line = file.readline()
    if not line: break
    wordlist = line.split('\t')
    if (wordlist[1].startswith('N')):
        stopword.append(wordlist[0])

print(stopword)

filename="datas.txt"  #파일 경로
file = open(filename, 'r', encoding="UTF-8")
kiwi=Kiwi()
kiwi.extract_add_words(reader.read)
kiwi.prepare()
stopwords = set(stopword)


def tokenize(sent): # 파일의 라인을 분석할 tokenize 함수
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용한다, 분석할때 나오는 결과에서 단어만 추출
    return [word
            for word, tag, _, _ in res
            if tag.startswith('N') and word not in stopwords] #불용어사전 적용
    



['것', '수', '이', '나', '사람', '등', '우리', '때', '년', '말', '일', '때문', '그것', '일', '문제', '사회', '중', '씨', '지금', '속', '하나', '집', '월', '데', '자신', '내', '경우', '명', '생각', '시간', '그녀', '앞', '번', '여자', '개', '전', '사실', '점', '정도', '원', '소리']


In [15]:
import tomotopy as tp

model = tp.LDAModel(k=10, alpha=0.1, eta=0.01, min_cf=20,min_df=10, tw=tp.TermWeight.PMI)
#LDA 모델을 적용해서 토픽 추출, k는 topic 개수
#alpha는 문헌-토픽 디리클레 분포의 하이퍼 파라미터
#eta는 토픽-단어 디리클레 분포의 하이퍼 파라미터 두개다 상수인듯하다
#min_cf는 단어의 최소 장서 빈도수, 전체 문헌내 출현빈도
#min_df는 단어의 최서 문헌 빈도수, 출현한 문헌 숫자수 의미
#tw는 용어 가중치 기법으로, ONE, IDF, PMI를 사용가능, ONE 보다는 PMI나 IDF 둘중 하나 사용 

for i, line in enumerate(open(filename, encoding='utf-8')): #해당 경로의 파일을 받아와 한 라인씩 model에 추가
    model.add_doc(tokenize(line)) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성
    if i % 10 == 0: print('Document #{} has been loaded'.format(i))
 
model.train(0) #학습 초기화

print('Total docs:', len(model.docs)) #문헌들
print('Total words:', model.num_words) #전체단어개수
print('Vocab size:', model.num_vocabs) #전체 어휘개수
 
 
for i in range(0,1000,20):
    print('Iteration {}\tLL per word: {}'.format(i, model.ll_per_word))
    model.train(20) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
  
for i in range(model.k): #k는 토픽의 개수
    res = model.get_topic_words(i, top_n=5) #하위 토픽 i에 해당하는 top_n개의 단어 반환
    print('Topic #{}'.format(i), end='\t')
    print(', '.join(w for w, p in res))


Document #0 has been loaded
Document #10 has been loaded
Document #20 has been loaded
Document #30 has been loaded
Document #40 has been loaded
Document #50 has been loaded
Document #60 has been loaded
Document #70 has been loaded
Document #80 has been loaded
Document #90 has been loaded
Document #100 has been loaded
Document #110 has been loaded
Document #120 has been loaded
Document #130 has been loaded
Document #140 has been loaded
Document #150 has been loaded
Document #160 has been loaded
Document #170 has been loaded
Document #180 has been loaded
Document #190 has been loaded
Document #200 has been loaded
Document #210 has been loaded
Document #220 has been loaded
Document #230 has been loaded
Document #240 has been loaded
Document #250 has been loaded
Document #260 has been loaded
Document #270 has been loaded
Document #280 has been loaded
Document #290 has been loaded
Document #300 has been loaded
Document #310 has been loaded
Document #320 has been loaded
Document #330 has bee